<a href="https://colab.research.google.com/github/bhkwon89/cp2/blob/main/cp2_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
situation_code = {
    1: '가족관계',
    2: '학업 및 진로',
    3: '학교폭력/따돌림',
    4: '대인관계',
    5: '연애,결혼,출산',
    6: '진로,취업,직장',
    7: '대인관계(부부,자녀)',
    8: '은퇴, 노후준비',
    9: '건강',
    10: '직작,업무,스트레스',
    11: '건강,죽음',
    12: '대인관계(노년)',
    13: '재정' 
}


emotion_code = {
    10: '분노',
    11: '툴툴대는',
    12: '좌절한',
    13: '짜증내는',
    14: '방어적인',
    15: '악의적인',
    16: '안달하는',
    17: '구역질나는',
    18: '노여워하는',
    19: '성가신',
    20: '슬픔',
    21: '실망한',
    22: '비통한',
    23: '후회되는',
    24: '우울한',
    25: '마비된',
    26: '염세적인',
    27: '눈물이 나는',
    28: '낙담한',
    29: '환멸을 느끼는',
    30: '불안',
    31: '두려운',
    32: '스트레스 받는',
    33: '취약한',
    34: '혼란스러운',
    35: '당혹스러운',
    36: '회의적인',
    37: '걱정스러운',
    38: '조심스러운',
    39: '초조한',
    40: '상처',
    41: '질투',
    42: '배신당한',
    43: '고립된',
    44: '충격 받은',
    45: '가난한, 불우한',
    46: '희생된',
    47: '억울한',
    48: '괴로워하는',
    49: '버려진',
    50: '당황',
    51: '고립된(당황한)',
    52: '남의 시선을 의식하는',
    53: '외로운',
    54: '열등감',
    55: '죄책감의',
    56: '부끄러운',
    57: '혐오스러운',
    58: '한심한',
    59: '혼란스러운',
    60: '기쁨',
    61: '감사하는',
    62: '신뢰하는',
    63: '편안한',
    64: '만족스러운',
    65: '흥분',
    66: '느긋',
    67: '안도',
    68: '신이 난',
    69: '자신하는'
}


In [5]:
!pip install gluonnlp pandas tqdm   
!pip install mxnet
!pip install sentencepiece
!pip install transformers
!pip install torch



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-5sv7t2ce/kobert-tokenizer_9d655b54f1f347eba03c2e358c0546ec
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-5sv7t2ce/kobert-tokenizer_9d655b54f1f347eba03c2e358c0546ec


In [10]:
pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-34pdgzzy
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-34pdgzzy
     |████████████████████████████████| 129 kB 14.0 MB/s 
     |████████████████████████████████| 54.7 MB 1.1 MB/s 
     |████████████████████████████████| 4.5 MB 54.4 MB/s 
     |████████████████████████████████| 1.2 MB 37.4 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.2 MB/s eta 0:00:41tcmalloc: large alloc 1147494400 bytes == 0x39d06000 @  0x7f6eec9ff615 0x58e046 0x4f2e5e 0x4d19df 0x51b31c 0x5b41c5 0x58f49e 0x51b221 0x5b41c5 0x58f49e 0x51837f 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4ba70a 0x538136 0x590055 0x51b180 0x5b41c5 0x58f49e 0x51837f 0x5b41c5 0x58f49e 0x51740e 0x58f2a7 0x517947 0x5b41c5 0x58f49e
     |████████████████████████████████| 881.9 MB

In [11]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from tqdm import tqdm, tqdm_notebook
from sklearn.model_selection import train_test_split

device = torch.device("cuda:0")

In [12]:
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel


tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')
tok = tokenizer.tokenize

# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

Downloading:   0%|          | 0.00/371k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/244 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/432 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/535 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [13]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):
   
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
         
    def __len__(self):
        return (len(self.labels))

In [14]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [15]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
def predict(sentence):
    dataset = [[sentence, '0']]
    test = BERTDataset(dataset, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(test, batch_size=batch_size, num_workers=2)
    model.eval()
    answer = 0
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        for logits in out:
            logits = logits.detach().cpu().numpy()
            answer = np.argmax(logits)
    return answer

In [16]:
train_set = pd.read_excel('/content/drive/MyDrive/cp2/dataset/감성대화말뭉치(최종데이터)_Training.xlsx')
validation_set = pd.read_excel('/content/drive/MyDrive/cp2/dataset/감성대화말뭉치(최종데이터)_Validation.xlsx')

In [17]:
train_set

,번호,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템응답1,사람문장2,시스템응답2,사람문장3,시스템응답3,사람문장4,시스템응답4
0,44164,청년,남성,"연애, 결혼, 출산",해당없음,기쁨,신이 난,아내가 드디어 출산하게 되어서 정말 신이 나.,아내분이 출산을 하시는군요. 정말 축하드려요.,아 지금 정말 신이 나.,잘 된 일이네요.,아기가 점점 클게 벌써 기대가 되네. 내가 많이 놀아줘야지.,좋은 아빠가 되실 거 같아요. 진심으로 축하드려요.,NaN,NaN
1,3926,노년,남성,"건강, 죽음",만성질환 유,불안,스트레스 받는,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.,약 종류가 많아 번거로우시겠어요.,건강할 때 관리 좀 잘할걸 하는 생각이 들더라고.,현재 상황에서 변화를 주기 위해 어떻게 하면 될까요?,약을 잘 챙겨 먹고 나을 수 있도록 노력해야지.,약을 잘 챙겨 먹어 건강해지시길 바랄게요.,NaN,NaN
2,50882,청소년,여성,학업 및 진로,해당없음,당황,당황,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.,고등학교 수업이 중학교 때와는 다르게 갑자기 어려워서 당황스러우시군요.,아직 학기 초인데 내가 수업에 잘 따라갈 수 있을지 걱정돼.,이 상황에서 할 수 있는 게 어떤 것이 있을까요?,중학교 수업을 들을 때보다 훨씬 더 열심히 할 거야.,수업을 어떤 방식으로 열심히 듣는게 좋을까요?,선생님이 강조하신 부분을 필기하고 집에서 매일 수업 내용을 복습하려고 해.,열심히 해서 좋은 결과가 있었으면 좋겠어요.
3,31303,노년,남성,재정,만성질환 무,기쁨,신이 난,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.,재취업 후 첫 월급이라 정말 기쁘시겠어요.,퇴직 후 다시는 돈을 못 벌 줄 알았는데 이렇게 월급으로 가족에게 맛있는 밥을 살 ...,이 기분을 유지하려면 어떻게 해야 할까요?,회사생활을 열심히 해서 계속 월급을 받을거야!,계속 원하시는 대로 회사생활이 이어지길 바랄게요.,NaN,NaN
4,47200,노년,여성,재정,만성질환 유,기쁨,안도,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.,기분 좋으시겠어요. 앞으로는 어떻게 하실 생각이신가요?,빚도 다 갚았으니 당분간은 아무 생각도 안 하며 살고 싶어.,빚도 다 갚았으니 당분간은 아무 생각도 안 하며 살고 싶으시군요.,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40874,50789,청년,남성,대인관계,해당없음,불안,혼란스러운,같이 사는 친구가 애완견을 데려왔는데 대부분 내가 돌보고 있어. 내가 주인인가 혼란...,친구 애완견을 대부분 혼자 돌보시는군요. 떠넘겨질까 봐 불안하시겠어요.,나한테 아예 떠넘길 거 같은데 친구한테 얘기해볼까 고민 중이야.,친구분의 생각이 궁금해 얘기해보려고 하시는군요.,이대로 내가 키우게 되면 안 되니까 확실하게 해야겠어.,그렇게 하셨을 떄 지금의 상황이 어떻게 되기를 바라시나요?,친구가 자기가 데려온 강아지에 대해 책임감을 갖고 스스로 잘 돌봤으면 좋겠어.,친구에게 얘기해서 혼자 부담을 떠안지 않으면 좋겠어요.
40875,48959,중년,남성,건강,해당없음,기쁨,안도,지난주에 건강검진 결과가 나왔는데 정상이라고 결과가 나왔어.,검진 결과가 좋게 나와서 안도하겠네요.,결과가 좋게 나와서 다행이야,건강검진에서 좋은결과를 받으신 상황이군요.,NaN,NaN,NaN,NaN
40876,36344,청소년,여성,가족관계,해당없음,슬픔,우울한,엄마는 내 꿈인 작가를 응원해 주고는 했는데 지금은 안 그래. 너무 슬퍼.,어머니의 응원이 사라져서 슬프시겠어요. 조금 더 자세히 말해주시겠어요?,내 재능이 남들보다 월등한 거는 아니라면서 취업해서 안정적으로 살았으면 좋겠다고 하셔.,이 상황에서 무엇을 시도해 보고 싶나요?,내가 다시 내 꿈을 어떻게 이룰 것인지 자세히 설명해 드려야겠어.,꿈을 어떻게 이룰 것인지 엄마에게 이야기해보려 하시는군요.,NaN,NaN
40877,14197,노년,여성,재정,만성질환 무,기쁨,감사하는,이렇게 좋은 운동 시설에서 경로 우대로 운동할 수 있다니 참 행운이야.,운동 시설을 쓸 수 있어서 기분이 좋으시겠어요.,덕분에 건강도 챙길 수 있고 너무 좋아. 안마기도 있던데 내일은 운동하고 안마기도 ...,운동할 때 안마기를 사용하면 어떻게 도움이 되나요?,운동으로 뭉친 근육을 풀어주는 것 같아. 그럼 덜 피로하겠지.,안마기로 피로가 많이 풀리시길 바라요.,NaN,NaN


In [18]:
train_set['문장'] = (train_set['사람문장1'] + ' ' + train_set['시스템응답1']  + ' ' + 
                   train_set['사람문장2'] + ' ' + train_set['시스템응답2'] + ' ' + 
                   train_set['사람문장3'] + ' ' + train_set['시스템응답3'] + 
                   train_set['사람문장4'] + ' ' + train_set['시스템응답4'])
validation_set['문장'] = (validation_set['사람문장1'] + ' ' + validation_set['시스템응답1']  + ' ' +
                        validation_set['사람문장2'] + ' ' + validation_set['시스템응답2'] + ' ' + 
                        validation_set['사람문장3'] + ' ' + validation_set['시스템응답3'] + ' ' +
                        validation_set['사람문장4'] + ' ' + validation_set['시스템응답4'] )

In [19]:

train_set = train_set.loc[:, ['감정_대분류', '문장']]
validation_set = validation_set.loc[:, ['감정_대분류', '문장']]

train_set.dropna(inplace=True)
validation_set.dropna(inplace=True)
train_set.columns = ['label', 'data']
validation_set.columns = ['label', 'data']

train_set.loc[(train_set['label'] == '기쁨'), 'label'] = 0
train_set.loc[(train_set['label'] == '기쁨 '), 'label'] = 0
train_set.loc[(train_set['label'] == '불안'), 'label'] = 1
train_set.loc[(train_set['label'] == '불안 '), 'label'] = 1
train_set.loc[(train_set['label'] == '당황'), 'label'] = 2
train_set.loc[(train_set['label'] == '슬픔'), 'label'] = 3
train_set.loc[(train_set['label'] == '분노'), 'label'] = 4
train_set.loc[(train_set['label'] == '상처'), 'label'] = 5
validation_set.loc[(validation_set['label'] == '기쁨'), 'label'] = 0                    
validation_set.loc[(validation_set['label'] == '불안'), 'label'] = 1                   
validation_set.loc[(validation_set['label'] == '당황'), 'label'] = 2                    
validation_set.loc[(validation_set['label'] == '슬픔'), 'label'] = 3                    
validation_set.loc[(validation_set['label'] == '분노'), 'label'] = 4                    
validation_set.loc[(validation_set['label'] == '상처'), 'label'] = 5   

In [20]:
train_set_data = [[i, str(j)] for i, j in zip(train_set['data'], train_set['label'])]
validation_set_data = [[i, str(j)] for i, j in zip(validation_set['data'], validation_set['label'])]

train_set_data, test_set_data = train_test_split(train_set_data, test_size = 0.2, random_state=4)

train_set_data = BERTDataset(train_set_data, 0, 1, tok, vocab, max_len, True, False)
test_set_data = BERTDataset(test_set_data, 0, 1, tok, vocab, max_len, True, False)
train_dataloader = torch.utils.data.DataLoader(train_set_data, batch_size=batch_size, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(test_set_data, batch_size=batch_size, num_workers=2)

In [ ]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

In [ ]:
torch.save(model, f'/content/drive/MyDrive/Colab Notebooks/SentimentAnalysisKOBert.pt')
torch.save(model.state_dict(), f'/content/drive/MyDrive/Colab Notebooks/SentimentAnalysisKOBert_StateDict.pt')